17 - 05 - 2023 h 13:51

## Things to install

In [1]:
# CLIP
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

# # REFCOCOG
!gdown 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
!tar -xf /content/refcocog.tar.gz
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

#LAVIS
!pip install salesforce-lavis


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8wfa_zo_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8wfa_zo_
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369370 sha256=a802989e1f55b061e74bf028faaa9494110fabefb1ab35605628880faa287672
  Stored in directory: /tmp/pip-ephem-wheel-cache-dbz1mtxk/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Downloading...
From: https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
To: /con

In [2]:
import clip
import json

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F

import numpy as np
import pandas
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import lavis
from lavis.common.gradcam import getAttMap
from lavis.models.blip_models.blip_image_text_matching import compute_gradcam
import locale
from typing import Sequence, Union
from scipy.ndimage import gaussian_filter

from scipy.ndimage import gaussian_filter

locale.getpreferredencoding = lambda: 'UTF-8'


In [3]:
blip_caption, image_preprocess_blip_cap, _ = lavis.models.load_model_and_preprocess(
    name='blip_caption', model_type='large_coco', is_eval=True, device='cuda'
    )

blip_im_text_match, image_preprocess_im_text_match, text_preprocess_im_text_match = lavis.models.load_model_and_preprocess(
    name="blip_image_text_matching", model_type="large", is_eval=True, device='cuda'
  )

modelCLIP, preprocessCLIP = clip.load("RN50", device='cuda')


100%|██████████| 1.66G/1.66G [00:16<00:00, 108MB/s]
100%|██████████| 3.43G/3.43G [00:43<00:00, 85.6MB/s]
100%|████████████████████████████████████████| 244M/244M [00:01<00:00, 137MiB/s]


## Functions to define

### Cosine similarity

In [20]:
def cosine_similarity(images_z: torch.Tensor, texts_z: torch.Tensor):
  # normalise the image and the text
  images_z /= images_z.norm(dim=-1, keepdim=True)
  texts_z /= texts_z.norm(dim=-1, keepdim=True)

  # evaluate the cosine similarity between the sets of features
  similarity = (texts_z @ images_z.T)

  return similarity.cpu()

### Rescale image

In [90]:
def rescale_image(image: Image, target_width: int = 720):
  w, h = image.size
  scale = target_width / w
  image = image.resize((int(w * scale), int(h * scale)))

  return image

### bbox_da_gradcam

In [97]:
def bbox_da_gradcam(
  gradcam, image: Image, text_tokens: torch.Tensor = None, average: bool = True, soglia=0.5
):

  image_norm = np.float32(image) / 255

  center = [0, 0, 0, 0]
  bbox = [0, 0, 0, 0]

  x, y, x_std, y_std = 0, 0, 0, 0

  if average:
    # get the average gradcap
    gradcam = getAttMap(image_norm, gradcam[0][1], blur=True)
    colour_map = gradcam - image_norm
    
    #--------------------------------------------------------
    h, w, c = colour_map.shape
    # # PRINT COLOUR MAP
    # fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    # ax.imshow(colour_map)
    # ax.axis('off')
    # fig.tight_layout()

    # # PRINT RED CHANNEL
    # red_channel = np.zeros((h, w, c))
    # for i in range(colour_map.shape[0]):
    #   for j in range(colour_map.shape[1]):
    #       red_channel[i][j][0] = colour_map[i][j][0]
    
    # fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    # ax.imshow(red_channel)
    # ax.axis('off')
    # fig.tight_layout()

    # # PRINT GREEEN CHANNEL
    # green_channel = np.zeros((h, w, c))
    # for i in range(colour_map.shape[0]):
    #   for j in range(colour_map.shape[1]):
    #     green_channel[i][j][0] = colour_map[i][j][1]
    
    # fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    # ax.imshow(green_channel)
    # ax.axis('off')
    # fig.tight_layout()
    
    # # PRINT BLUE CHANNEL
    # blue_channel = np.zeros((h, w, c))
    # for i in range(colour_map.shape[0]):
    #   for j in range(colour_map.shape[1]):
    #     blue_channel[i][j][0] = colour_map[i][j][2]
    
    # fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    # ax.imshow(blue_channel)
    # ax.axis('off')
    # fig.tight_layout()

    #---------------------------------------------------------------------------

    xs = []
    ys = []
    
    for i in range(colour_map.shape[0]):
      for j in range(colour_map.shape[1]):
        if colour_map[i][j][0] > soglia:
          xs.append(i)
          ys.append(j)

    if len(xs)==0:
      print('NOT FOUND!')
    else:
      x_center = int(np.median(xs))
      y_center = int(np.median(ys))
      x_std = np.std(xs)
      y_std = np.std(ys)

      center = [x_center, y_center, x_std, y_std]
      bbox = [int(x_center-30), int(y_center-30), int(x_center+30), int(y_center+30)]

  return center, bbox


### sub_region

In [91]:
def sub_region(image, target, modelCLIP, preprocessCLIP):
  '''
  Args:
  ..............................
  Return:
  ..............................
  '''

  # CANDIDATE SUB-REGIONS
  half_images = [image]
  A, B = image.size
  for i in range(3):
    im = image.crop((A/4*i,0,A/4*i+A/2,B))
    half_images.append(im)

  for j in range(3):
    im = image.crop((0,B/4*j,A,B/4*j+B/2))
    half_images.append(im)

  # IMAGE FRAGMENTS
  image_pecies = []
  for j in range(3):
    for i in range(3):
      im = image.crop((A/4*i,B/4*j,A/4*i+A/2,B/4*j+B/2))
      image_pecies.append(im)

  # ENCODING
  images = [preprocessCLIP(im) for im in image_pecies]
  images = torch.tensor(np.stack(images)).cuda()

  with torch.no_grad():
    images_z = modelCLIP.encode_image(images).float()

  #TARGET
  target_caption = clip.tokenize(target).cuda()

  with torch.no_grad():
      target_caption_z = modelCLIP.encode_text(target_caption).float()
  
  # IMAGE SIMILARITIES
  im_sim = torch.Tensor(0)

  for im in images_z:
    similarity = cosine_similarity(im, target_caption_z)
    im_sim = torch.cat((im_sim, similarity),0)

  topk_similarities, topk_idx = torch.topk(im_sim, 5, sorted=True)
  for i in range(5):
    if topk_similarities[i]<0.15 :
      topk_idx = topk_idx[0:i]
      break

  if topk_similarities[0] > 0.20:
    return image_pecies[topk_idx[0]], topk_idx[0]+10

  common_idx = topk_idx.tolist()
  key = 0
  if all(x in common_idx for x in [0, 1, 2]):
    key = 1
  elif all(x in common_idx for x in [3, 4, 5]):
    key = 2
  elif all(x in common_idx for x in [6, 7, 8]):
    key = 3
  elif all(x in common_idx for x in [0, 3, 6]):
    key = 4
  elif all(x in common_idx for x in [1, 4, 7]):
    key = 5
  elif all(x in common_idx for x in [2, 5, 8]):
    key = 6

  return half_images[key], key


### Dataset

In [10]:
class RefcocogDataset(Dataset):
    def __init__(self, base_path, split=None, transform=None, tokenization=None):
        annotation_path = base_path + "/annotations/"

        self.IMAGES_PATH = base_path + "/images/"
        self.transform = transform
        self.tokenization = tokenization

        tmp_annotations = pandas.read_pickle(annotation_path + "refs(umd).p")
        tmp_instances = json.load(open(annotation_path + "instances.json", "r"))

        annotations_dt = pandas.DataFrame.from_records(tmp_annotations) \
            .filter(items=["image_id", "split", "sentences", "ann_id"])

        instances_dt = pandas.DataFrame.from_records(tmp_instances['annotations'])

        self.annotations = annotations_dt \
            .merge(instances_dt[["id", "bbox", "area"]], left_on="ann_id", right_on="id") \
            .drop(columns="id")

        if split is not None:
            self.annotations = self.__get_annotations_by_split(split.lower())

    def getImage(self, sample):
        id = sample['idx'][0].item()
        item = self.annotations.iloc[id]
        image = self.__getimage(item.image_id)

        return image

    def getSentences(self, sample):
        id = sample['idx'][0].item()
        item = self.annotations.iloc[id]

        return self.__extract_sentences(item.sentences)
    
    def showImage(self, train_features, train_bbox):
        img = self.getImage(train_features)
        img1 = ImageDraw.Draw(img)
        img1.rectangle([(train_bbox[0].item(), train_bbox[1].item()), (train_bbox[2].item(), train_bbox[3].item())], outline ="red")
        img.show()

    def __get_annotations_by_split(self, split):
        return self.annotations[self.annotations.split == split].reset_index()

    def __getimage(self, id):
        return Image.open(self.IMAGES_PATH + "COCO_train2014_" + str(id).zfill(12) + ".jpg")

    def __extract_sentences(self, sentences):
        return [f"a photo of {s['sent']}" for s in sentences]

    def __tokenize_sents(self, sentences):
        return [self.tokenization(s) for s in sentences]

    def __len__(self):
        return self.annotations.shape[0]

    def __getitem__(self, idx):
        item = self.annotations.iloc[idx]
        image = self.__getimage(item.image_id)
        sentences = self.__extract_sentences(item.sentences)

        if self.transform:
            image = self.transform(image)

        if self.tokenization:
            sentences = self.__tokenize_sents(sentences)

        sample = {'idx': idx, 'image': image, 'sentences': sentences}

        return sample, item.bbox

### Compute intersection, accuracy, precision

In [98]:
def computeIntersection(bbox, BBOX):
  x_min = bbox[0]
  y_min = bbox[1]
  x_max = bbox[2]
  y_max = bbox[3]
  X_MIN = BBOX[0]
  Y_MIN = BBOX[1]
  X_MAX = BBOX[2]
  Y_MAX = BBOX[3]

  dx = min(x_max, X_MAX) - max(x_min, X_MIN)
  dy = min(y_max, Y_MAX) - max(y_min, Y_MIN)
  if (dx>=0) and (dy>=0):
      area = dx*dy
  else:
      area = 0
  return area

def computeAccuracy(bbox, BBOX):
    intersection = computeIntersection(bbox, BBOX)
    area1 = (bbox[2]-bbox[0])*(bbox[3]-bbox[1])
    area2 = (BBOX[2]-BBOX[0])*(BBOX[3]-BBOX[1])
    union = area1 + area2 - intersection
    return intersection / union

def computePrecision(bbox, BBOX):
    intersection = computeIntersection(bbox, BBOX)
    area1 = (bbox[2]-bbox[0])*(bbox[3]-bbox[1]) + 0.000001
    return intersection / area1


### Print bbox

In [81]:
def print_bbox(image, bbox):

  image_box = image.copy()
  plt.figure(figsize=(10,10))

  for i in range(bbox[0], bbox[2]):
    for j in range(bbox[1], bbox[3]):
      image_box.putpixel((j, i), (225, 0, 0))

  plt.imshow(image_box)


## Instantion of the dataset

In [13]:
REFCOCOG_PATH = "refcocog"

train_dataset = RefcocogDataset(REFCOCOG_PATH, split="train", transform=preprocessCLIP, tokenization=clip.tokenize)
val_dataset = RefcocogDataset(REFCOCOG_PATH, split="val", transform=preprocessCLIP, tokenization=clip.tokenize)
test_dataset = RefcocogDataset(REFCOCOG_PATH, split="test", transform=preprocessCLIP, tokenization=clip.tokenize)

train_dataloader = DataLoader(train_dataset, shuffle=False)
val_dataloader = DataLoader(val_dataset, shuffle=False)
test_dataloader = DataLoader(test_dataset, shuffle=False)

## Baseline

In [99]:
modelCLIP.cuda().eval()
device = 'cuda'

def baseline(loader, dataset, modelCLIP, preprocessCLIP, soglia=0.5):

  n_samples = 0
  tot_accuracy = 0
  tot_precision = 0
  tot_accuracy1 = 0
  tot_precision1 = 0

  for data_features, data_bbox in loader:

    sentences = dataset.getSentences(data_features)
    target = sentences[0]
    target_bbox = [0, 0, 0, 0]
    target_bbox[0] = int(data_bbox[1].item())
    target_bbox[1] = int(data_bbox[0].item())
    target_bbox[2] = int(data_bbox[1].item() + data_bbox[3].item())
    target_bbox[3] = int(data_bbox[2].item() + data_bbox[0].item()) 

    image = dataset.getImage(data_features)        
    w, h = image.size
    
    # GRADCAM VISUALIZATION OF THE WHOLE IMAGE
    input_text = text_preprocess_im_text_match["eval"](target)
    input_image = image_preprocess_im_text_match["eval"](image).unsqueeze(0).cuda()
    text_tokens = blip_im_text_match.tokenizer(input_text, return_tensors="pt").to('cuda')
    gradcam, _ = compute_gradcam(
        blip_im_text_match, input_image, input_text, text_tokens, block_num=7
      )
    
    center, bbox = bbox_da_gradcam(gradcam, image, text_tokens=text_tokens, average=True, soglia=soglia)

    precision = computePrecision(bbox, target_bbox)
    accuracy = computeAccuracy(bbox, target_bbox)

    
    # FIND THE RIGHT SUBREGION
    restricted_image, key = sub_region(image, target, modelCLIP, preprocessCLIP)
    
    # GRADCAM VISUALIZATION OF THE SUB-REGION
    input_image = image_preprocess_im_text_match["eval"](restricted_image).unsqueeze(0).cuda()
    text_tokens = blip_im_text_match.tokenizer(input_text, return_tensors="pt").to('cuda')
    gradcam, _ = compute_gradcam(
        blip_im_text_match, input_image, input_text, text_tokens, block_num=7
      )
    
    center_restricted, bbox_restricted = bbox_da_gradcam(gradcam, restricted_image, text_tokens=text_tokens, average=True, soglia=soglia)

    # print_bbox(image, bbox)
    # print_bbox(restricted_image, bbox_restricted)

    if key==0:
      print('key=0')

    elif key==1:
      print('key=1')
    elif key==2:
      print('key=2')
      bbox_restricted[1] += int(w/4)
      bbox_restricted[3] += int(w/4)
    elif key==3:
      print('key=3')
      bbox_restricted[1] += int(w/2)
      bbox_restricted[3] += int(w/2)
    elif key==4:
      print('key=4')
    elif key==5:
      print('key=5')
      bbox_restricted[0] += int(h/4)
      bbox_restricted[2] += int(h/4)
    elif key==6:
      print('key=6')
      bbox_restricted[0] += int(h/2)
      bbox_restricted[2] += int(h/2)

    elif key==10:
      print('key=10')
    elif key==11:
      print('key=13')
      bbox_restricted[1] += int(w/4)
      bbox_restricted[3] += int(w/4)      
    elif key==12:
      print('key=16')
      bbox_restricted[1] += int(w/2)
      bbox_restricted[3] += int(w/2)
    elif key==13:
      print('key=11')
      bbox_restricted[0] += int(h/4)
      bbox_restricted[2] += int(h/4)
    elif key==14:
      print('key=14')
      bbox_restricted[1] += int(w/4)
      bbox_restricted[3] += int(w/4)  
      bbox_restricted[0] += int(h/4)
      bbox_restricted[2] += int(h/4)
    elif key==15:
      print('key=17')
      bbox_restricted[1] += int(w/2)
      bbox_restricted[3] += int(w/2)  
      bbox_restricted[0] += int(h/4)
      bbox_restricted[2] += int(h/4)
    elif key==16:
      print('key=12')
      bbox_restricted[0] += int(h/2)
      bbox_restricted[2] += int(h/2)
    elif key==17:
      print('key=15')      
      bbox_restricted[1] += int(w/4)
      bbox_restricted[3] += int(w/4)  
      bbox_restricted[0] += int(h/2)
      bbox_restricted[2] += int(h/2)
    elif key==18:
      print('key=18')
      bbox_restricted[1] += int(w/2)
      bbox_restricted[3] += int(w/2)  
      bbox_restricted[0] += int(h/2)
      bbox_restricted[2] += int(h/2)

    # print_bbox(image, bbox_restricted)
    
    precision1 = computePrecision(bbox_restricted, target_bbox)
    accuracy1 = computeAccuracy(bbox_restricted, target_bbox)
    
    n_samples += 1
    tot_accuracy += accuracy
    tot_precision += precision
    tot_accuracy1 += accuracy1
    tot_precision1 += precision1

    print('\n---------------------------------------------------------------')
    print(f'IMAGE: {n_samples+1}')
    print('Precision:', precision, 'Accuracy:', accuracy, '\tPrecision:', precision1, 'Accuracy:', accuracy1)


    if n_samples%10==0:
      print('TOT_Precision:', tot_precision/n_samples, 'TOT_Accuracy:', tot_accuracy/n_samples, '\tTOT_Precision:', tot_precision1/n_samples, 'TOT_Accuracy:', tot_accuracy1/n_samples)
  
  return #tot_accuracy/n_samples
  

## Run the baseline

In [101]:
baseline(test_dataloader, test_dataset, modelCLIP, preprocessCLIP, soglia=0.5)

key=17

---------------------------------------------------------------
IMAGE: 2
Precision: 0.9999999997222222 Accuracy: 0.1310425160163075 	Precision: 0.9999999997222222 Accuracy: 0.1310425160163075
key=10

---------------------------------------------------------------
IMAGE: 3
Precision: 0.9999999997222222 Accuracy: 0.024581768521679755 	Precision: 0.883333333087963 Accuracy: 0.021651800912371486
NOT FOUND!
key=18

---------------------------------------------------------------
IMAGE: 4
Precision: 0.9166666664120371 Accuracy: 0.1402107409925221 	Precision: 0.0 Accuracy: 0.0
NOT FOUND!
key=10

---------------------------------------------------------------
IMAGE: 5
Precision: 0.0 Accuracy: 0.0 	Precision: 0.9999999997222222 Accuracy: 0.1360544217687075
key=17

---------------------------------------------------------------
IMAGE: 6
Precision: 0.9999999997222222 Accuracy: 0.2566478933485421 	Precision: 0.9999999997222222 Accuracy: 0.2566478933485421
key=12

---------------------------

RuntimeError: ignored